### Работа с изображениями

In [1]:
from PIL import Image, Draw
import pandas as pd

### Кожа

In [2]:
img0 = Image.open('skin.jpg')
img1 = Image.open('skin1.jpg')
img2 = Image.open('skiin644.jpg')
images = [img0, img1, img2]


In [3]:
def dataset_create(images):
    pixels = []
    for img in images:
        h = img.size[1]
        w = img.size[0]
        pix = img.load() # выгрузка пикселей
        
        for i in range(h):
            for k in range(w):
                p = pix[k, i]
                pixels.append(p)
    df = pd.DataFrame(pixels)
    df.drop_duplicates(inplace = True)
    df.rename(columns= {0:'r', 1:'g', 2:'b'}, inplace = True)
    return df

In [4]:
df_skin = dataset_create(images)

In [5]:
df_skin['is_skin'] = 1

In [6]:
df_skin

,r,g,b,is_skin
0,219,206,171,1
1,215,202,167,1
2,209,200,161,1
3,203,191,153,1
4,202,189,154,1
...,...,...,...,...
1057685,186,102,68,1
1057686,183,99,65,1
1057690,178,98,65,1
1057754,157,80,36,1


### Не кожа

In [7]:
img0 = Image.open('photo1.jpg')
img1 = Image.open('photo2.jpg')
images = [img0, img1]


In [8]:
df_not_skin = dataset_create(images)
df_not_skin['is_skin'] = 0
df_not_skin.head()

,r,g,b,is_skin
0,66,162,238,0
8,67,163,239,0
16,66,162,236,0
24,65,161,235,0
43,67,163,237,0


In [9]:
""" оффтоп
# если тысяча фото
# dictionary = {}
for i in range(1000):
    name = 'photo' + i +'.jpg'
    dictionary[name] = Image.open(name)
"""

" оффтоп\n# если тысяча фото\n# dictionary = {}\nfor i in range(1000):\n    name = 'photo' + i +'.jpg'\n    dictionary[name] = Image.open(name)\n"

### Объединение

In [10]:
df = df_skin.append(df_not_skin)

df

,r,g,b,is_skin
0,219,206,171,1
1,215,202,167,1
2,209,200,161,1
3,203,191,153,1
4,202,189,154,1
...,...,...,...,...
1173417,31,129,28,0
1173504,0,47,7,0
1173505,18,80,15,0
1173545,5,104,22,0


### Построение модели

In [11]:
from sklearn.model_selection import train_test_split
x = df.drop('is_skin', axis = 1)
y = df['is_skin']

x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, random_state = 1)

In [12]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB() # наивный байес
nb.fit(x_tr, y_tr)

GaussianNB()

In [13]:
pred = nb.predict(x_ts)
#nb.predict_proba(x_ts) # вероятности 1 или 0

In [14]:
from sklearn.metrics import accuracy_score

accuracy_score(pred, y_ts)

0.8804470398711236

In [15]:
from sklearn.model_selection import cross_val_score

cross_val_score(nb, x, y, cv = 10) # каждое значение - accuracy каждого перемешанного датафрейма

array([0.89365183, 0.87640959, 0.96642167, 0.93936267, 0.93344744,
       0.84605316, 0.66821889, 0.77887133, 0.94391865, 0.89093335])

### RGB to HSV

In [16]:
import colorsys

hsv_pixels = []
for i in range(len(df)):
    r = df.iloc[i, 0]
    g = df.iloc[i, 1]
    b = df.iloc[i, 2]
    hsv_pix = colorsys.rgb_to_hsv(r, g, b)
    hsv_pixels.append(hsv_pix)
    
df_hsv = pd.DataFrame(hsv_pixels)    

In [17]:
df_hsv.head()

,0,1,2
0,0.121528,0.219178,219
1,0.121528,0.223256,215
2,0.135417,0.229665,209
3,0.126667,0.246305,203
4,0.121528,0.237624,202


In [18]:
df_hsv['is_skin'] = df['is_skin'].values

In [19]:
df_hsv.head()

,0,1,2,is_skin
0,0.121528,0.219178,219,1
1,0.121528,0.223256,215,1
2,0.135417,0.229665,209,1
3,0.126667,0.246305,203,1
4,0.121528,0.237624,202,1


In [20]:
x = df_hsv.drop('is_skin', axis = 1)
y = df_hsv['is_skin']

cross_val_score (GaussianNB(), x, y, cv = 10) # почти все оценки выше 90%

array([0.90296516, 0.8617348 , 0.9983387 , 0.97626359, 0.96015405,
       0.90643878, 0.77187374, 0.8897503 , 0.97903242, 0.8990888 ])

### ДЗ
- Проверить пиксели собаки и не собаки (научиться определять собаку)
- Почитать про функцию Draw